In [1]:
import requests
import json
import re
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
nltk.download('stopwords')


ModuleNotFoundError: No module named 'requests'

In [12]:
def getReceiptInfo(imageFile):
    receiptOcrEndpoint = 'https://ocr.asprise.com/api/v1/receipt' # Receipt OCR API endpoint
    r = requests.post(receiptOcrEndpoint, data = { \
      'client_id': 'TEST',        # Use 'TEST' for testing purpose \
      'recognizer': 'auto',       # can be 'US', 'CA', 'JP', 'SG' or 'auto' \
      'ref_no': 'ocr_python_123', # optional caller provided ref code \
      }, \
        files = {"file": open(imageFile, "rb")})
    print(r.text)
    return r.text

In [23]:
info = getReceiptInfo("./tesco_cake_receipt.jpg")
print(info)

{
  "ocr_type" : "receipts",
  "request_id" : "P_87.75.27.53_loc0lhn4_zx1",
  "ref_no" : "ocr_python_123",
  "file_name" : "tesco_cake_receipt.jpg",
  "request_received_on" : 1698616818832,
  "success" : true,
  "image_width" : 640,
  "image_height" : 480,
  "image_rotation" : -1.550,
  "recognition_completed_on" : 1698616819189,
  "receipts" : [ {
    "merchant_name" : "Durhan Market Place Express",
    "merchant_address" : "Durhan Market Place Express",
    "merchant_phone" : null,
    "merchant_website" : null,
    "merchant_tax_reg_no" : null,
    "merchant_company_reg_no" : null,
    "region" : null,
    "mall" : null,
    "country" : "GB",
    "receipt_no" : "A0000000041010",
    "date" : "2023-10-23",
    "time" : "11:41",
    "items" : [ {
      "amount" : 1.40,
      "category" : null,
      "description" : "Tesco British Double Cream",
      "flags" : "",
      "qty" : 1,
      "remarks" : null,
      "tags" : null,
      "unitPrice" : null
    }, {
      "amount" : 2.30,
   

In [38]:
def getItemList(info):
    regex = '[0-9.+/\£$()]'
    single_char_regex = '(\\b[a-z] \\b|\\b [a-z]\\b)'
    info_json = json.loads(info)
    items_json = info_json['receipts'][0]['items']
    description_list = []
    for item in items_json:
        description = item['description']
        description = re.sub(regex, "", description).lower()
        description = re.sub(single_char_regex, "", description)
        print(description)
        if description not in description_list:
            description_list.append(description)
    
    # Remove duplicates
    return description_list
        
    

In [36]:
def removeStopWords(description_list):
      stop_words = set(stopwords.words('english'))
      filtered_sentence = [item for item in description_list if not item in stop_words]
      return filtered_sentence

In [39]:
description_list = getItemList(info)
description_list = removeStopWords(description_list)
print(description_list)

tesco british double cream
tesco strawberries
tesco chocolate sponge cake
tesco dark cooking chocolate
['tesco british double cream', 'tesco strawberries', 'tesco chocolate sponge cake', 'tesco dark cooking chocolate']
